In [1]:
import numpy as np
import open3d as o3d
import torch

# from sam2.build_sam import build_sam2
# from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator

from superprimitive_fusion.scanner import capture_spherical_scans
from superprimitive_fusion.utils import bake_uv_to_vertex_colours
from superprimitive_fusion.mesh_fusion_utils import get_mesh_components, show_mesh_boundaries
from superprimitive_fusion.mesh_fusion import fuse_meshes

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# device0 = torch.device("cuda:0")

# torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
# if torch.cuda.get_device_properties(0).major >= 8:
#     torch.backends.cuda.matmul.allow_tf32 = True
#     torch.backends.cudnn.allow_tf32 = True

In [3]:
# sam2_checkpoint = "../models/SAM2/sam2.1_hiera_large.pt"
# model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

# sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device0, apply_postprocessing=False)

# # mask_generator = SAM2AutomaticMaskGenerator(sam2)
# mask_generator = SAM2AutomaticMaskGenerator(
#     model=sam2,
#     points_per_side=64,
#     points_per_batch=128,
#     pred_iou_thresh=0.7,
#     stability_score_thresh=0.92,
#     stability_score_offset=0.7,
#     crop_n_layers=1,
#     box_nms_thresh=0.7,
#     crop_n_points_downscale_factor=2,
#     min_mask_region_area=25.0,
#     use_m2m=True,
# )

In [4]:
mesh = o3d.io.read_triangle_mesh("../data/power-drill/textured.obj", enable_post_processing=True)

bake_uv_to_vertex_colours(mesh)

mesh.compute_vertex_normals()

bb = mesh.get_minimal_oriented_bounding_box()
scale = np.mean(bb.get_max_bound())

In [6]:
from superprimitive_fusion.mesh_fusion_utils import sanitise_mesh, colour_transfer, count_inconsistent_normal_pairs
from superprimitive_fusion.debug_utils import debug_mesh
import pymeshfix

scans = capture_spherical_scans(
    mesh=mesh,
    mask_generator=None,
    num_views=6,
    radius=0.3,
    width_px=360,
    height_px=240,
    fov=70,
    dropout_rate=0,
    depth_error_std=0.0,
    translation_error_std=0,
    rotation_error_std_degs=0,
    k=10,
    sampler="fibonacci",
)
meshes = [scan['mesh'] for scan in scans]

# o3d.visualization.draw_geometries(
#     meshes,
#     window_name="Virtual scan",
#     front=[0.3, 1, 0],
#     lookat=[0, 0, 0],
#     up=[0, 0, 1],
#     zoom=0.7,
# )

In [7]:
j = 6
repaired = meshes[0]
for i in range(1,j):
    fused_mesh = fuse_meshes(
        repaired,
        meshes[i],
        h_alpha=3,
        trilat_iters=2,
        shift_all=False,
        fill_holes=False,
    )
    V0 = np.asarray(fused_mesh.vertices)
    F0 = np.asarray(fused_mesh.triangles)
    C0 = np.asarray(fused_mesh.vertex_colors)

    V0, F0 = sanitise_mesh(V0, F0)

    meshfix = pymeshfix.MeshFix(V0, F0)
    meshfix.repair(verbose=False, joincomp=False, remove_smallest_components=False)
    V1, F1 = meshfix.points, meshfix.faces

    C1 = colour_transfer(V0, C0, V1)

    repaired = o3d.geometry.TriangleMesh()
    repaired.vertices = o3d.utility.Vector3dVector(V1)
    repaired.triangles = o3d.utility.Vector3iVector(F1)
    repaired.vertex_colors = o3d.utility.Vector3dVector(C1)
    repaired.compute_vertex_normals()
    # repaired.compute_face_normals()

    o3d.visualization.draw_geometries([repaired])


o3d.visualization.draw_geometries(
    [fused_mesh],
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

In [ ]:
print('start stats')
print("edge manifold:", fused_mesh.is_edge_manifold(allow_boundary_edges=True))
print("vertex manifold:", fused_mesh.is_vertex_manifold())
print("self-intersecting:", fused_mesh.is_self_intersecting())
print("watertight:", fused_mesh.is_watertight())

import trimesh
V = np.asarray(fused_mesh.vertices)
F = np.asarray(fused_mesh.triangles)
tm = trimesh.Trimesh(vertices=V, faces=F, process=False)
print("winding consistent:", tm.is_winding_consistent)
print("watertight:", tm.is_watertight)
print("euler number:", tm.euler_number)  # sanity: wildly negative often = non-manifold mess
print('finish stats')

o3d.visualization.draw_geometries(
    [fused_mesh],
    window_name="Virtual scan",
    front=[0.3, 1, 0],
    lookat=[0, 0, 0],
    up=[0, 0, 1],
    zoom=0.7,
)

new_points = np.asarray(fused_mesh.vertices)
fused_mesh_triangles = np.asarray(fused_mesh.triangles)
new_colours = np.asarray(fused_mesh.vertex_colors)

V0, F0 = sanitise_mesh(new_points, fused_mesh_triangles)
C0 = new_colours

# meshfix = pymeshfix.MeshFix(V0, F0)
# meshfix.repair(verbose=True, joincomp=False, remove_smallest_components=False)
# V1, F1 = meshfix.points, meshfix.faces

mf = pymeshfix.PyTMesh(True)
mf.load_array(V0, F0)
# mf.fill_small_boundaries(nbe=5)
mf.clean()
# mf.join_closest_components()
V1, F1 = mf.return_arrays()

# mf = pymeshfix.PyTMesh(False)
# mf.load_array(V0, F0)

# mf.fill_small_boundaries(nbe=100)

# V1, F1 = mf.return_arrays()

C1 = colour_transfer(V0, C0, V1)

repaired = o3d.geometry.TriangleMesh()
repaired.vertices = o3d.utility.Vector3dVector(V1)
repaired.triangles = o3d.utility.Vector3iVector(F1)
repaired.vertex_colors = o3d.utility.Vector3dVector(C1)
repaired.compute_vertex_normals()

o3d.visualization.draw_geometries([repaired])

components = get_mesh_components(fused_mesh, show=True)

# n_components = len(components)
# tris_added = len(F1) - len(fused_mesh_triangles)
# n_inconsistent_pairs_holes = count_inconsistent_normal_pairs(fused_mesh, show=False)
# n_inconsistent_pairs_filled = count_inconsistent_normal_pairs(repaired, show=False)
# show_mesh_boundaries(fused_mesh, show=True, edges=True)
# show_mesh_boundaries(repaired, show=True, edges=True)
# show_mesh_boundaries(repaired, show=True, edges=True, base_mesh=fused_mesh)

# print(f'{tris_added} triangles were added to fill holes')
# print(f'{n_inconsistent_pairs_holes} (w/ holes) is the number of triangle neighbours with inconsistent normals')
# print(f'{n_inconsistent_pairs_filled} (holes filled) is the number of triangle neighbours with inconsistent normals')